In [1]:
from langchain_community.document_loaders import UnstructuredURLLoader

In [2]:
urls = ['https://www.victoriaonmove.com.au', 'https://www.victoriaonmove.com.au/index.html', 'https://www.victoriaonmove.com.au/contact.html']
loader = UnstructuredURLLoader(urls=urls)
data = loader.load()

In [3]:
data

[Document(metadata={'source': 'https://www.victoriaonmove.com.au'}, page_content="Loading...\n\nRelocate with confidence\n\nYour trusted partner in seamless moving and packing solutions!\n\nGoogle logo G\n\nGoogle Rating\n\nGold star icon\n\nGold star icon\n\nGold star icon\n\nGold star icon\n\nGold star icon\n\n5 stars, 111 reviews\n\n\n\n\n\n\n\n\n\nRequst A call for You:\n\nExplore Our Fleet\n\nVehicles Tailored for Your Moving Needs\n\nIcon\n\nSmall Truck\n\nWith 2 Mover\n\nFrom $110/hr\n\n4.5 ton 20 cubic Our Small Truck can manage a student apartment, small office .\n\nIcon\n\nMedium Truck\n\nWith 2 Mover\n\nFrom $130 p/hr\n\n6 ton 30-35 cubic\n\nOur Medium trucks can manage 2-3 bedroom with single lounge outdoor boxes and miscellaneous.\n\nIcon\n\nLarge Truck\n\nWith 2 Mover\n\nFrom $140 p/hr\n\n8 ton 40 cubic Our Large trucks can handle 3-4 bedrooms with double lounge outdoor boxes and miscellaneous.\n\nIcon\n\nX-Large Truck\n\nWith 2 Mover\n\nFrom $150 p/hr\n\n4.5 ton 20 cubic

# Performing Chunk Operations

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Split data
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)
docs = text_splitter.split_documents(data)
print("Total Number of documents: ", len(docs))

Total Number of documents:  11


In [5]:
docs[0]

Document(metadata={'source': 'https://www.victoriaonmove.com.au'}, page_content='Loading...\n\nRelocate with confidence\n\nYour trusted partner in seamless moving and packing solutions!\n\nGoogle logo G\n\nGoogle Rating\n\nGold star icon\n\nGold star icon\n\nGold star icon\n\nGold star icon\n\nGold star icon\n\n5 stars, 111 reviews\n\n\n\n\n\n\n\n\n\nRequst A call for You:\n\nExplore Our Fleet\n\nVehicles Tailored for Your Moving Needs\n\nIcon\n\nSmall Truck\n\nWith 2 Mover\n\nFrom $110/hr\n\n4.5 ton 20 cubic Our Small Truck can manage a student apartment, small office .\n\nIcon\n\nMedium Truck\n\nWith 2 Mover\n\nFrom $130 p/hr\n\n6 ton 30-35 cubic\n\nOur Medium trucks can manage 2-3 bedroom with single lounge outdoor boxes and miscellaneous.\n\nIcon\n\nLarge Truck\n\nWith 2 Mover\n\nFrom $140 p/hr\n\n8 ton 40 cubic Our Large trucks can handle 3-4 bedrooms with double lounge outdoor boxes and miscellaneous.\n\nIcon\n\nX-Large Truck\n\nWith 2 Mover\n\nFrom $150 p/hr\n\n4.5 ton 20 cubic 

In [6]:
from langchain.vectorstores import Chroma
from langchain_chroma import Chroma
from langchain.schema import Document
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_google_genai import GoogleGenerativeAI
import os

In [7]:
from dotenv import load_dotenv
load_dotenv()

True

In [8]:
vectorstore = Chroma.from_documents(documents=docs, embedding=GoogleGenerativeAIEmbeddings(
        model="models/embedding-001",
        GOOGLE_API_KEY = "GOOGLE_API_KEY"
))

In [9]:
retriver = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [10]:
retrived_docs = retriver.invoke("What kind of service they provide?")

In [11]:
len(retrived_docs)

3

In [12]:
retrived_docs

[Document(id='11ea7295-0590-4c7a-98f1-588159c31c35', metadata={'source': 'https://www.victoriaonmove.com.au/index.html'}, page_content="2 BED HOME:\n\n2 Beds, Mattress, Tall Boy, Bedside, Refrigerator, Washing Machine,Dresser, Barbeque, Some Boxes, Loose stuff.\n\nBig HOME:\n\n4-5 Bed Room, Double Lounge, Mattress, Tall Boy, Bedside, Refrigerator, Washing Machine,Dresser, Barbeque, Some Boxes,Furniture, Loose stuff. Get a Quote\n\nInterstate Removalists:\n\nSeamless Relocation Across State Lines\n\n\n\nMelbourne to Sydney Removalists\n\nBook Now\n\n\n\nMelbourne to Brisbane Removalists\n\nBook Now\n\n\n\nMelbourne to Adelaide Removalists\n\nBook Now\n\n\n\nMelbourne to Canberra Removalists\n\nBook Now\n\n\n\nOUR MOVING WORK & ETHICS\n\nMoving house furniture is an art that comes with experience and a steadfast commitment to our customers' needs. It's a challenging industry, and not everyone survives for long without the expertise and dedication that we bring to every job. Our long hist

In [13]:
print(retrived_docs[0].page_content)

2 BED HOME:

2 Beds, Mattress, Tall Boy, Bedside, Refrigerator, Washing Machine,Dresser, Barbeque, Some Boxes, Loose stuff.

Big HOME:

4-5 Bed Room, Double Lounge, Mattress, Tall Boy, Bedside, Refrigerator, Washing Machine,Dresser, Barbeque, Some Boxes,Furniture, Loose stuff. Get a Quote

Interstate Removalists:

Seamless Relocation Across State Lines



Melbourne to Sydney Removalists

Book Now



Melbourne to Brisbane Removalists

Book Now



Melbourne to Adelaide Removalists

Book Now



Melbourne to Canberra Removalists

Book Now



OUR MOVING WORK & ETHICS

Moving house furniture is an art that comes with experience and a steadfast commitment to our customers' needs. It's a challenging industry, and not everyone survives for long without the expertise and dedication that we bring to every job. Our long history as Removalists in Melbourne is a testament to our sincerity and hard work.


# Now Connceting to the LLm Models

In [14]:
llm = GoogleGenerativeAI(model='gemini-1.5-pro', temperature=0, max_tokens=500)

In [15]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistance for question-answering tasks. "
    "use the following pieces of retrived context to answer "
    "the question. if you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [16]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriver, question_answer_chain)

In [17]:
response = rag_chain.invoke({"input": "What kind of services they provide?"})
print(response["answer"])

They provide interstate removalist services, specializing in moves originating from Melbourne.  They offer moving and packing solutions with various truck sizes to accommodate different needs, from small apartments to larger homes.  Their pricing varies depending on truck size, starting from $110/hr for a small truck.
